In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim

In [2]:
import unicodedata
import string

all_letters = string.ascii_letters + "_- .,;'0123456789"

# Turn a Unicode string to plain ASCII, thanks to http://stackoverflow.com/a/518232/2809427
def unicode_to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

print(unicode_to_ascii('Ślusàrski'))

Slusarski


In [3]:
# Add padding '<PAD>' to the lookup table for all_letters
# char 2 index
char2idx = {c:i+1 for i, c in enumerate(all_letters)}


# index 2 char
index2char = {i+1:c for i,c in enumerate(all_letters)}


print("number of letters: %i \n" %len(char2idx))

number of letters: 69 



In [4]:
words = ['four','word5','AaBbCc','example','eightlet', 'ninelette','tenletters', 'elevenlette','twelveletter']

In [5]:
max_len = max([len(word) for word in words])
print(max_len)

12


In [6]:
word_idx = []

for i, word in enumerate(words):
    word_idx.append([char2idx[c] for c in word])
    print(len(word_idx[i]))

4
5
6
7
8
9
10
11
12


In [17]:
from numpy.random import randn
y = [y//2 for y in randn(len(words))]

y = torch.FloatTensor(y).long()
print(y)
    


 0
 0
 0
 0
-1
 0
 0
-1
 0
[torch.LongTensor of size 9]



In [18]:
class CNN(nn.Module):
    def __init__(self, input_size, output_size):
        super(CNN, self).__init__()
        
        self.embedding = nn.Embedding(input_size, len(char2idx))
        self.input_size = input_size
        
        self.conv = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(2,len(char2idx)))
        self.pool = nn.AdaptiveMaxPool2d((3,1), return_indices=False)
        
        '''self.sequential = nn.Sequential(nn.Conv2d(in_channels=1, out_channels=1, 
                                                  kernel_size=(2,len(vocab))),
                                        nn.MaxPool2d(kernel_size=(2,len(vocab))))'''
        self.linear = nn.Linear(1, output_size) # n_channels, output_size
        
    def forward(self, x):
        
        print('\nn_length:',x.size())
        
        x = self.embedding(x)
        print('embed:',x.size())
        
        # x = torch.cat((embedded_x, i), dim=1)
        
        x = x.unsqueeze(0).unsqueeze(0)
        print('unsqueezed:',x.size())
        
        x = self.conv(x)
        print('conv_x:',x.size())
        x = self.pool(x)
        print('pool:',x.size())
        
        ''' for i in range((self.input_size-4)//2):
            x = self.sequential(x)
            print('conv2:',x.size())
        '''
        x = self.linear(x)
        print('output:',x.size())
        
        return x

In [19]:
cnn = CNN(len(char2idx), 2)

In [20]:
criterion = nn.NLLLoss() #cross entropy loss = log softmax + NLL loss
optimizer = optim.Adam(cnn.parameters())

def train(model, criterion, optimizer, x, y):
    
    # reset gradient
    optimizer.zero_grad()

    # forward pass
    fx = model.forward(x)
    
    # get the loss
    loss = criterion(fx, y)

    # backward pass
    loss.backward()

    # update parameters
    optimizer.step()

    # return the actual loss data, not the Variable
    return loss.data[0]

for i, word in enumerate(word_idx):
    loss = 0.
    name_var = Variable(torch.LongTensor(word))
    indexes = [i for i in range(len(name_var))]
    x = torch.LongTensor(indexes).float()
    x = x.unsqueeze(1)
    index_var = Variable(x)
    print(name_var.size())
    cnn(name_var)
    loss += train(cnn, criterion, optimizer, x, y)

torch.Size([4])

n_length: torch.Size([4])
embed: torch.Size([4, 69])
unsqueezed: torch.Size([1, 1, 4, 69])
conv_x: torch.Size([1, 1, 3, 1])
pool: torch.Size([1, 1, 3, 1])
output: torch.Size([1, 1, 3, 2])

n_length: torch.Size([4, 1])


TypeError: torch.index_select received an invalid combination of arguments - got ([32;1mtorch.FloatTensor[0m, [32;1mint[0m, [31;1mtorch.FloatTensor[0m), but expected (torch.FloatTensor source, int dim, torch.LongTensor index)

In [16]:
torch.zeros(1,1,3,18)


(0 ,0 ,.,.) = 
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
[torch.FloatTensor of size 1x1x3x18]